In [127]:
import quandl
import pandas as pd
import requests
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import json
import datetime as dt
import json
import pandas as pd
from datetime import datetime

In [125]:
test = requests.get("http://www.quandl.com/api/v3/datasets/YALE/SPCOMP?start_date=2018-12-20&end_date=2018-12-31&api_key=KZv99Dx6pP9M_zVY9SzW")

In [5]:
# Federal Debt: Total Public Debt as Percent of Gross Domestic Product, Quarterly
# Units: Percent of GDP, Seasonally Adjusted
# Source: FRED (Federal Reserve Bank St. Louis)

debt = quandl.get("FRED/GFDEGDQ188S", authtoken="ckJvd5G2a2dkK9nun3xS")

In [6]:
# S&P 500 Index Composite, Monthly
# Units: Index
# Source: Yale University, Online Data Robert Shiller

sp500 = quandl.get("YALE/SPCOMP", authtoken="ckJvd5G2a2dkK9nun3xS")

In [7]:
# Unemployment Rate, Monthly
# Units: Percent, Seasonally Adjusted
# Source: FRED (Federal Reserve Bank St. Louis)

u = quandl.get("FRED/M0892AUSM156SNBR", authtoken="ckJvd5G2a2dkK9nun3xS")

In [8]:
# Balance on Current Account, NIPA's, Quarterly
# Units: Billions of Dollars Seasonally Adjusted Annual Rate
# Source: FRED (Federal Reserve Bank St. Louis)

balance = quandl.get("FRED/NETFI", authtoken="ckJvd5G2a2dkK9nun3xS")

In [9]:
presidents = pd.read_csv('us_presidents.csv')


In [14]:
date_start_series = list(presidents['start'])

In [161]:
# Converting presidents first dates in office into a datetime object

new_date_start_series = []
for row in date_start_series:
    date = dt.datetime.strptime(row,'%B %d, %Y').strftime('%Y-%m-%d')
    datetime_object = datetime.strptime(date, '%Y-%m-%d')
    new_date_start_series.append(datetime_object)

In [18]:
date_end_series = list(presidents['end'])

In [25]:
date_end_series[-2] = 'January 20, 2017'

In [26]:
date_end_series[-1] = 'November 12, 2019'

In [156]:
# Converting presidents final dates in office into a datetime object

new_date_end_series = []
for row in date_end_series:
    date = dt.datetime.strptime(row,'%B %d, %Y').strftime('%Y-%m-%d')
    datetime_object = datetime.strptime(date, '%Y-%m-%d')
    new_date_end_series.append(datetime_object)

In [131]:
datetime_object = datetime.strptime(new_date_end_series[0], '%Y-%m-%d')

In [72]:
presidents['start_date'] = new_date_start_series 

In [73]:
presidents['end_date'] = new_date_end_series

In [75]:
new_president_df = presidents[['president', 'start_date', 'end_date']].copy()

In [76]:
new_president_df.head()

,president,start_date,end_date
0,George Washington,1789-04-30,1797-03-04
1,John Adams,1797-03-04,1801-03-04
2,Thomas Jefferson,1801-03-04,1809-03-04
3,James Madison,1809-03-04,1817-03-04
4,James Monroe,1817-03-04,1825-03-04


In [71]:
sp500.head()

,S&P Composite,Dividend,Earnings,CPI,Long Interest Rate,Real Price,Real Dividend,Real Earnings,Cyclically Adjusted PE Ratio
Year,,,,,,,,,
1871-01-31,4.44,0.26,0.4,12.464061,5.320000,91.257956,5.343934,8.221438,NaN
1871-02-28,4.50,0.26,0.4,12.844641,5.323333,89.750706,5.185596,7.977841,NaN
1871-03-31,4.61,0.26,0.4,13.034972,5.326667,90.602080,5.109879,7.861352,NaN
1871-04-30,4.74,0.26,0.4,12.559226,5.330000,96.685821,5.303442,8.159141,NaN
1871-05-31,4.86,0.26,0.4,12.273812,5.333333,101.438813,5.426768,8.348873,NaN


In [107]:
sp = sp500.iloc[:,:1]

In [108]:
# Get the date in a new column to be able to merge tables on Date
sp['Date'] = sp.index

/Users/carterbouley/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [109]:
# Adding a new column with t-1 (lag) values to calculate monthly returns on S&P
sp['lag'] = sp['S&P Composite'].shift(1)

/Users/carterbouley/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [110]:
to_drop = sp.iloc[:1]
sp.drop(to_drop.index, axis=0, inplace=True)

/Users/carterbouley/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [111]:
# The S&P monthly returns is simply the rate of change between months ((t - t-1)/t-1)*100

sp['monthly_return_per'] = ((sp['S&P Composite']-sp['lag'])/sp['lag'])*100

/Users/carterbouley/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [115]:
sp.head()
type(sp['Date'][0]

pandas._libs.tslibs.timestamps.Timestamp

In [90]:
to_drop  = new_president_df[:17].index

In [92]:
new_president_df = new_president_df.drop(to_drop, axis = 0)

In [97]:
type(new_president_df.iloc[0].start_date)

str

In [162]:
presidents['start_datetime'] = new_date_start_series

In [163]:
presidents['end_datetime'] = new_date_end_series

In [164]:
presidents.head()

,Unnamed: 0,S.No.,start,end,president,prior,party,vice,start_date,end_date,start_datetime,end_datetime
0,0,1,"April 30, 1789","March 4, 1797",George Washington,Commander-in-Chief of the Continental Army ...,Nonpartisan [13],John Adams,1789-04-30,1797-03-04,1789-04-30,1797-03-04
1,1,2,"March 4, 1797","March 4, 1801",John Adams,1st Vice President of the United States,Federalist,Thomas Jefferson,1797-03-04,1801-03-04,1797-03-04,1801-03-04
2,2,3,"March 4, 1801","March 4, 1809",Thomas Jefferson,2nd Vice President of the United States,Democratic- Republican,Aaron Burr,1801-03-04,1809-03-04,1801-03-04,1809-03-04
3,3,4,"March 4, 1809","March 4, 1817",James Madison,5th United States Secretary of State (1801–...,Democratic- Republican,George Clinton,1809-03-04,1817-03-04,1809-03-04,1817-03-04
4,4,5,"March 4, 1817","March 4, 1825",James Monroe,7th United States Secretary of State (1811–...,Democratic- Republican,Daniel D. Tompkins,1817-03-04,1825-03-04,1817-03-04,1825-03-04


In [169]:
sp.head()

,S&P Composite,Date,lag,monthly_return_per
Year,,,,
1871-02-28,4.50,1871-02-28,4.44,1.351351
1871-03-31,4.61,1871-03-31,4.50,2.444444
1871-04-30,4.74,1871-04-30,4.61,2.819957
1871-05-31,4.86,1871-05-31,4.74,2.531646
1871-06-30,4.82,1871-06-30,4.86,-0.823045


In [260]:
# Indexing each dataframe by year/month


presidents['month'] = presidents['end_datetime'].dt.month
presidents['year'] = presidents['end_datetime'].dt.year
presidents['monthyear'] = presidents['year'].map(str) + '/' + presidents['month'].map(str)

sp['month'] = sp['Date'].dt.month
sp['year'] = sp['Date'].dt.year
sp['monthyear'] = sp['year'].map(str) + '/' + sp['month'].map(str)

In [239]:
# new dataframe, merged from old dataframes on date

df = sp1.merge( p1, on='monthyear', how='left')

In [241]:
# Selecting only columns we want

new_df = df[['Date','monthly_return_per', 'president_y']]

In [246]:
# filling in all presidents 

new_df['president'] = new_df['president_y'].fillna(method='backfill')

/Users/carterbouley/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [257]:
# filling in Donald trump (as there was no endate)

new_df.president.fillna('Donald Trump', inplace = True)

/Users/carterbouley/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [268]:
new_df.groupby('president').mean()

,monthly_return_per
president,
Barack Obama,1.064475
Benjamin Harrison,0.080491
Bill Clinton,1.217110
Calvin Coolidge,1.760395
Chester A. Arthur,-0.805555
Donald Trump,0.800276
Dwight D. Eisenhower,0.905926
Franklin D. Roosevelt,0.772067
George H. W. Bush,0.926910


In [270]:
new_df.tail()

,Date,monthly_return_per,president_y,president
1778,2019-04-30,3.559940,NaN,Donald Trump
1779,2019-05-31,-1.690543,NaN,Donald Trump
1780,2019-06-30,1.242158,NaN,Donald Trump
1781,2019-07-31,3.665528,NaN,Donald Trump
1782,2019-08-31,-3.742519,NaN,Donald Trump
